In [49]:
import numpy as np
import sys
sys.path.insert(0, '/Users/jmmoon/Documents/GitHub/sparse_kmeans/experiment/22_05_03_2024')
import sqlite3
from datetime import datetime
import pandas as pd
con = sqlite3.connect('/Users/jmmoon/Documents/GitHub/sparse_kmeans/experiment/22_05_03_2024/db_050324_early_stopping.db')
cursor_db = con.cursor()
thres = 0.01

def iter_criterion_activated(table_pd, criterion, thres):
    value_vec = table_pd[criterion]
    pct_change_abs = value_vec.pct_change().abs()
    iter_stop = pct_change_abs.lt(thres).idxmax()
    
    return(iter_stop)

In [79]:
pd.Series([0,0,0,0]).idxmax()

0

In [7]:
cursor_db = con.cursor()
cursor = con.execute(
		"""
        SELECT *
        FROM sparse_kmeans_isee_clean
        WHERE rep=200
		""")
table = cursor.fetchall()
table_pd = pd.DataFrame(table)
table_pd.columns = ["rep", "iter", "sep", "dim", "rho", "sparsity", 
            "stop_og", "stop_sdp", "stop_loop",           
             "acc", "obj_prim", "obj_dual", "obj_original",
             "true_pos", "false_pos",  "false_neg",
             "diff_x_tilde_fro", "diff_x_tilde_op", "diff_x_tilde_ellone",
             "time_est", "time_SDP", "jobdate",           
            "survived_indices", "cluster_est"
            ]

In [50]:
iter_criterion_activated(table_pd, "obj_original", thres)
iter_criterion_activated(table_pd, "obj_prim", thres)

19

In [91]:
def early_stop(table_pd, criterion, patience_param):
    patience = patience_param
    max_so_far = abs(table_pd[criterion][1])
    for i, row in table_pd.iterrows():
        if i >1:
            #print(f"iter:{i}")
            value_vec = table_pd[criterion][0:i+1].abs()
            max_now = value_vec.max()
            #print(max_now)
            #print(max_so_far)
            if max_now > max_so_far:
                max_so_far = max_now
                patience = 3
            #    print(f"patience rest to {patience_param}")
            else:
                patience = patience - 1
            #    print(f"patience:{patience}")
            if patience == 0:
                iter_stop = i-patience_param
            #    print(f"stopped at: {iter_stop}")
                break
    if patience>0:
        iter_stop = 0
    return(iter_stop)
    

In [119]:
iter_stop_table = pd.DataFrame({"rep":[], "iter_stop_pct_change_og":[], "iter_stop_pct_change_prim":[], "iter_stop_earlystop_og":[], "iter_stop_earlystop_prim":[] })
iter_stop_table

,rep,iter_stop_pct_change_og,iter_stop_pct_change_prim,iter_stop_earlystop_og,iter_stop_earlystop_prim


In [116]:
iter_stop_table

,rep,iter_stop_pct_change_og,iter_stop_earlystop_og,iter_stop_earlystop_prim


In [120]:

iter_stop_table = pd.DataFrame({"rep":[], "iter_stop_pct_change_og":[], "iter_stop_pct_change_prim":[], "iter_stop_earlystop_og":[], "iter_stop_earlystop_prim":[] })
for i in range(1):
    rep = i+1
    cursor_db = con.cursor()
    cursor = con.execute(
		"""
        SELECT *
        FROM sparse_kmeans_isee_clean
        WHERE rep=?
		""", [rep])
    table = cursor.fetchall()
    table_pd = pd.DataFrame(table)
    table_pd.columns = ["rep", "iter", "sep", "dim", "rho", "sparsity", 
            "stop_og", "stop_sdp", "stop_loop",           
             "acc", "obj_prim", "obj_dual", "obj_original",
             "true_pos", "false_pos",  "false_neg",
             "diff_x_tilde_fro", "diff_x_tilde_op", "diff_x_tilde_ellone",
             "time_est", "time_SDP", "jobdate",           
            "survived_indices", "cluster_est"
            ]
    iter_stop_pct_change_og = iter_criterion_activated(table_pd, "obj_original", thres)
    iter_stop_pct_change_prim = iter_criterion_activated(table_pd, "obj_prim", thres)
    iter_stop_earlystop_og = early_stop(table_pd, "obj_original", patience_param)
    iter_stop_earlystop_prim = early_stop(table_pd, "obj_prim", patience_param)
    iter_stop_table.loc[i, :] = [rep, iter_stop_pct_change_og, iter_stop_pct_change_prim,  iter_stop_earlystop_og, iter_stop_earlystop_prim]
    


iter:2
849.7079829421408
849.7079829421408
patience:2
iter:3
849.7079829421408
849.7079829421408
patience:1
iter:4
896.7734123925334
849.7079829421408
patience rest to 3
iter:5
896.7734123925334
896.7734123925334
patience:2
iter:6
896.7734123925334
896.7734123925334
patience:1
iter:7
896.7734123925334
896.7734123925334
patience:0
stopped at: 4
iter:2
7.7130226789818295
7.7130226789818295
patience:2
iter:3
7.7130226789818295
7.7130226789818295
patience:1
iter:4
7.7130226789818295
7.7130226789818295
patience:0
stopped at: 1


In [121]:
iter_stop_table

,rep,iter_stop_pct_change_og,iter_stop_pct_change_prim,iter_stop_earlystop_og,iter_stop_earlystop_prim
0,1.0,21.0,11.0,4.0,1.0


In [113]:
table_pd["obj_original"][1:30]

1      849.707983
2      740.648108
3      763.281181
4      896.773412
5      621.624241
6      520.175118
7      756.336499
8      894.935628
9      853.518019
10     713.991125
11     759.895988
12     625.954127
13     529.418528
14     899.422254
15     762.010026
16    1041.006658
17    2054.753416
18    2241.353753
19    2112.238143
20    2189.022194
21    2186.608079
22    2148.348321
23    2174.375900
24    2162.329914
25    2199.328789
26    2170.689478
27    2155.433343
28    2228.128098
29    2255.768292
Name: obj_original, dtype: float64

In [114]:
table_pd[1:20]

,rep,iter,sep,dim,rho,sparsity,stop_og,stop_sdp,stop_loop,acc,...,false_pos,false_neg,diff_x_tilde_fro,diff_x_tilde_op,diff_x_tilde_ellone,time_est,time_SDP,jobdate,survived_indices,cluster_est
1,1,1,4.0,400,0.45,10,0,0,0,0.544,...,53,10,185.493261,40.072920,147.569156,14.104160,53.282676,2024-05-02 00:30:15.000000,"15,23,53,55,57,68,69,71,76,82,83,98,103,108,11...","1,1,1,2,1,1,2,2,1,1,1,1,2,1,1,1,1,2,2,1,1,1,1,..."
2,1,2,4.0,400,0.45,10,0,0,0,0.544,...,49,10,186.418382,38.735672,150.184759,10.769887,53.922448,2024-05-02 00:30:15.000000,"20,47,52,53,54,55,56,57,68,82,97,98,102,103,11...","1,2,1,1,1,1,2,1,1,2,2,1,2,2,1,1,1,2,2,2,2,1,2,..."
3,1,3,4.0,400,0.45,10,0,0,0,0.516,...,46,10,185.403547,38.917513,146.305819,10.673281,48.867598,2024-05-02 00:30:15.000000,"20,43,56,57,67,68,81,82,97,98,102,132,133,134,...","2,1,1,2,2,2,1,1,1,1,1,2,1,2,1,2,1,1,1,1,2,1,2,..."
4,1,4,4.0,400,0.45,10,0,0,1,0.520,...,35,10,190.173198,38.533714,154.856496,10.536827,58.484514,2024-05-02 00:30:15.000000,"47,57,68,81,82,96,97,101,102,121,134,151,157,1...","2,2,2,2,2,2,2,2,2,1,1,2,1,2,2,2,2,1,1,2,2,2,2,..."
5,1,5,4.0,400,0.45,10,0,0,0,0.518,...,31,9,183.243364,38.561325,146.946961,10.537768,55.916234,2024-05-02 00:30:15.000000,"7,15,82,83,97,98,100,101,102,103,114,134,151,1...","1,1,2,1,1,2,2,1,2,2,1,2,2,1,1,2,2,2,2,2,1,2,1,..."
6,1,6,4.0,400,0.45,10,0,0,0,0.542,...,28,8,191.141570,38.467340,152.673715,10.763243,56.474685,2024-05-02 00:30:15.000000,"2,7,15,24,53,54,81,82,134,150,151,171,172,173,...","2,2,1,2,1,1,1,2,1,1,1,1,2,2,1,2,2,1,1,2,1,1,2,..."
7,1,7,4.0,400,0.45,10,0,0,0,0.612,...,28,7,185.121801,38.255307,151.382516,11.166735,112.074391,2024-05-02 00:30:15.000000,"1,2,7,14,15,16,23,24,25,53,54,64,133,134,135,1...","1,1,1,2,1,2,2,2,1,2,1,1,1,1,2,2,1,1,2,1,2,1,1,..."
8,1,8,4.0,400,0.45,10,0,0,0,0.594,...,41,4,184.571970,35.909471,146.797368,10.322357,56.836629,2024-05-02 00:30:15.000000,"1,2,4,6,7,8,20,27,53,61,64,65,80,124,126,127,1...","2,1,2,1,2,1,1,2,2,1,1,2,2,2,2,2,2,2,2,2,1,2,2,..."
9,1,9,4.0,400,0.45,10,0,0,0,0.626,...,38,3,184.052345,37.608455,144.357906,10.425652,53.281724,2024-05-02 00:30:15.000000,"1,2,3,6,7,8,9,20,24,52,53,90,123,124,125,126,1...","2,1,1,1,1,1,1,1,1,2,1,1,2,1,1,2,1,2,1,2,1,2,2,..."
10,1,10,4.0,400,0.45,10,0,0,0,0.586,...,34,5,186.105563,36.868396,150.796066,10.672277,59.102163,2024-05-02 00:30:15.000000,"1,2,5,6,9,20,23,24,25,51,52,58,94,123,124,125,...","1,2,1,2,1,1,1,1,1,1,1,2,1,1,1,2,1,1,2,2,2,1,1,..."


In [2]:

	
	priv_lev_vec = cursor.fetchall()
	priv_lev_vec_np = np.array(priv_lev_vec).flatten()
	print(priv_lev_vec_np)
	for priv_lev in priv_lev_vec_np:
		cursor = con.execute(
			"""
			SELECT
				avg(p_val<0.05) as power,
				mechanism||statistic as method,
				sample_size
			FROM ldp_disc_basic_comparison
			WHERE
				dim = ? and
				priv_lev = ?
			GROUP BY statistic, mechanism, sample_size
			""",
			[k, priv_lev])
		table = cursor.fetchall()
		table_pd = pd.DataFrame(table)
		table_pd.columns = ["power", "method", "n"]
		table_pd = table_pd.pivot(index= "n", columns = "method", values = "power")
		table_pd.to_csv("purturbunif_power_og_k"+str(k)+ "_priv"+ str(int(10*priv_lev)) +".csv")



[0.5 1.  2. ]
[0.5 1.  2. ]
[2.  1.  0.5]
